<a href="https://colab.research.google.com/github/maharshoaib786/Auto-Trading-Bot/blob/main/MT5_Bot_Limit_Orders_(v110_7_Capped_TPs_Preserved).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mt5_grid_martingale.py  (v110.7-CappedTPsPreserved)
"""
Grid bot for MetaTrader 5 using a predefined lot sequence and dynamic, multi-grid hedging.
This version adds a configurable slippage setting for all market orders,
handles "Invalid Price" errors for limit orders by retrying with price adjustment,
and implements specific behaviors when a grid side is dynamically capped.

• Lot Sizing:
    - Initial hedge uses LOT_SMALL.
    - Grid trades use a predefined list: GRID_LOT_SEQUENCE.
• Multi-Grid Hedging (Internal Capping & New Grid):
    - When a grid's (Grid A) position count hits DYNAMIC_POSITIONS_TRIGGER, that side is "capped".
    - Capped side of Grid A:
        - Stops opening new positions (limit orders and re-hedges).
        - Existing positions on the capped side have their comments updated (e.g., "GridA_CappedBuy").
        - TPs and SLs of existing opened positions on the capped side remain unchanged.
        - These capped positions are excluded from TP synchronization.
    - An internal counter-hedge market order is placed on the *opposite side within the same Grid A*.
        - The volume of this hedge is typically related to the 50% of total lot volume of the capped side.
        - The Stop Loss for this counter-hedge is set to the Take Profit of the last position
          in Grid A's capped side.
        - This counter-hedge position itself has its Take Profit set to 0.0.
    - After Grid A's internal counter-hedge is successfully placed, a new independent grid (Grid B)
      is created IF the number of active grids is less than MAX_ACTIVE_GRIDS.
    - Grid B starts with its own standard initial hedge (BUY and SELL LOT_SMALL orders).
• Independent Grids: Each grid (identified by a unique magic number and name) is managed independently.
• Reset on Closure: If an open position closes on an *uncapped* side of a grid, all pending orders
  for that grid's side are cancelled, and the lot sequence for that grid's side restarts.
  Capped sides do not reset or re-hedge.
• Configurable via .env.
"""
from __future__ import annotations
import os, sys, time, math, logging, traceback
import MetaTrader5 as mt5
from dotenv import load_dotenv

# Ensure UTF-8 encoding on Windows
if hasattr(sys.stdout, "reconfigure"):
    sys.stdout.reconfigure(encoding="utf-8")

load_dotenv()


# --- Configuration ---
SYMBOL                    = os.getenv("SYMBOL", "XAUUSDc")
LOT_SMALL                 = float(os.getenv("LOT_SMALL", 0.01))
LOT_MAX                   = float(os.getenv("LOT_MAX", 20))
GRID_LOT_SEQUENCE         = [0.01, 0.02, 0.02, 0.03, 0.05, 0.07, 0.10, 0.14, 0.20, 0.29, 0.42, 0.60, 0.86, 1.23, 1.76, 2.53, 3.62, 5.20, 7.46, 10.70]
GRID_STEP_PIPS            = float(os.getenv("GRID_STEP_PIPS", 80))
DYNAMIC_STEP_PIPS         = float(os.getenv("DYNAMIC_STEP_PIPS", 90))
PROFIT_PIPS               = float(os.getenv("PROFIT_PIPS", 350))
DYNAMIC_PROFIT_PIPS       = float(os.getenv("DYNAMIC_PROFIT_PIPS", 600))
DYNAMIC_POSITIONS_TRIGGER = int(os.getenv("DYNAMIC_POSITIONS_TRIGGER", 27))
MAX_POSITIONS             = int(os.getenv("MAX_POS", 60)) # Max total open positions across ALL grids
PROFIT_TARGET_AMT         = float(os.getenv("PROFIT_TARGET_AMT", 1000))
OPEN_DELAY                = float(os.getenv("OPEN_DELAY", 2))
LOOP_MS                   = int(os.getenv("LOOP_MS", 100))
LOG_FILE                  = os.getenv("LOG_FILE", "grid_bot.log")
LOGIN                     = int(os.getenv("MT5_LOGIN", 0))
PASSWORD                  = os.getenv("MT5_PASSWORD", "")
SERVER                    = os.getenv("MT5_SERVER", "")
PENDING_ORDER_EXPIRATION_MIN = int(os.getenv("PENDING_ORDER_EXPIRATION_MIN", 0))
MAX_PENDING_GRID_ORDERS_PER_SIDE = int(os.getenv("MAX_PENDING_GRID_ORDERS_PER_SIDE", 1))
LOG_BALANCE_INTERVAL      = int(os.getenv("LOG_BALANCE_INTERVAL", 100))
MAX_ACTIVE_GRIDS          = int(os.getenv("MAX_ACTIVE_GRIDS", 2)) # Limit for concurrent grids
INVALID_PRICE_RETRY_LIMIT = int(os.getenv("INVALID_PRICE_RETRY_LIMIT", 10))
SLIPPAGE                  = int(os.getenv("SLIPPAGE", 10)) # Slippage for market orders
INVALID_PRICE_ADJUST_PIPS = float(os.getenv("INVALID_PRICE_ADJUST_PIPS", 40.0)) # Pips to adjust price by on invalid price retry


# --- Logging setup ---
log = logging.getLogger("grid_bot")
log.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(message)s")

console_handler = logging.StreamHandler(sys.stdout)
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(formatter)
log.addHandler(console_handler)

file_handler = logging.FileHandler(LOG_FILE, mode="w", encoding="utf-8")
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)
log.addHandler(file_handler)

log.propagate = False


# --- State variables ---
initial_equity: float      = 0.0
grid_states = {}
base_magic_number = 12345
next_magic_number = 12346
loop_counter: int          = 0

# --- Utilities ---
def mt5_login():
    global initial_equity
    log.info("Attempting to initialize MetaTrader 5...")
    if not mt5.initialize(login=LOGIN, password=PASSWORD, server=SERVER, timeout=10000):
        log.error(f"MT5 initialize() failed, error code = {mt5.last_error()}")
        mt5.shutdown()
        sys.exit(1)

    log.info(f"MetaTrader 5 initialized successfully. Terminal version: {mt5.version()}")
    account_info = mt5.account_info()
    if account_info is None:
        log.error(f"Failed to get account info, error code = {mt5.last_error()}")
        mt5.shutdown()
        sys.exit(1)
    log.info(f"Logged in to account: {account_info.login}, Server: {account_info.server}, Balance: {account_info.balance:.2f} {account_info.currency}")


    log.info(f"Selecting symbol: {SYMBOL}")
    if not mt5.symbol_select(SYMBOL, True):
        log.error(f"Symbol {SYMBOL} not found or not enabled in Market Watch, error code = {mt5.last_error()}")
        all_symbols = mt5.symbols_get()
        if all_symbols and any(s.name == SYMBOL for s in all_symbols if s is not None):
            log.info(f"Symbol {SYMBOL} exists but was not selected. Attempting to re-select.")
        else:
            log.warning(f"Symbol {SYMBOL} does not appear to be available on the broker.")
        mt5.shutdown()
        sys.exit(1)

    symbol_info = mt5.symbol_info(SYMBOL)
    if symbol_info is None:
        log.error(f"Failed to get info for symbol {SYMBOL}, error code = {mt5.last_error()}")
        mt5.shutdown()
        sys.exit(1)
    log.info(f"Symbol {SYMBOL} selected. Description: {symbol_info.description}, Digits: {symbol_info.digits}")

    if initial_equity == 0.0:
        initial_equity = account_info.equity
        log.info(f"✅ Login successful. Account: {LOGIN}, Initial Equity for profit tracking set to: {initial_equity:.2f} {account_info.currency}")
    else:
        log.info(f"✅ Re-login successful. Account: {LOGIN}, Current Equity: {account_info.equity:.2f} {account_info.currency}. Profit tracking continues from {initial_equity:.2f}.")


def pip_val() -> float:
    info = mt5.symbol_info(SYMBOL)
    if not info:
        log.critical(f"CRITICAL: Could not get symbol info for {SYMBOL} in pip_val. Cannot determine pip value. Exiting.")
        raise ValueError(f"Failed to get symbol info for {SYMBOL}, cannot determine pip_val.")

    if info.digits == 5 or info.digits == 4:
        return 0.0001
    elif info.digits == 3 or info.digits == 2:
        return 0.01
    else:
        log.warning(f"Uncommon number of digits ({info.digits}) for {SYMBOL}. Using point value as pip value: {info.point}")
        return info.point

def tp_price(direction: str, entry_price: float, pips_to_use: float) -> float:
    price_offset = pips_to_use * pip_val()
    s_info_digits = mt5.symbol_info(SYMBOL).digits

    if direction == "BUY":
        return round(entry_price + price_offset, s_info_digits)
    else: # SELL
        return round(entry_price - price_offset, s_info_digits)

def adjust_lot(lot: float) -> float:
    info = mt5.symbol_info(SYMBOL)
    if not info:
        log.error(f"Could not get symbol info for {SYMBOL} in adjust_lot. Returning original lot.")
        return lot

    volume_step = getattr(info, "volume_step", 0.01)
    volume_min = getattr(info, "volume_min", 0.01)
    volume_max = getattr(info, "volume_max", LOT_MAX)

    if volume_step <= 0:
        log.warning(f"Volume step for {SYMBOL} is invalid ({volume_step}). Using 0.01.")
        volume_step = 0.01

    lot = round(lot / volume_step) * volume_step

    lot = max(volume_min, lot)
    lot = min(volume_max, lot)
    lot = min(LOT_MAX, lot)

    precision = 0
    if volume_step > 0:
        precision = abs(int(math.log10(volume_step))) if volume_step < 1 else 0

    return round(lot, precision)

def format_mt5_comment(grid_name: str, action: str) -> str:
    sane_action = "".join(filter(str.isalnum, action))
    if not sane_action:
        sane_action = "trade"

    comment = f"Grid{grid_name}_{sane_action}"
    return comment[:31]

# --- Trade execution (Market Orders) ---
def send_market_order(direction: str, lot: float, magic: int, grid_name: str, action_comment_str: str, stop_loss: float = 0.0, take_profit: float | None = None) -> bool:
    mt5_comment = format_mt5_comment(grid_name, action_comment_str)

    adjusted_lot = adjust_lot(lot)
    symbol_info_vol_min = getattr(mt5.symbol_info(SYMBOL), "volume_min", 0.01)
    if adjusted_lot < symbol_info_vol_min :
        log.warning(f"Market Order: Adjusted lot {adjusted_lot} is below minimum {symbol_info_vol_min} for {direction} {lot} ({mt5_comment}). Order not sent.")
        return False

    price = 0.0
    calculated_tp = 0.0

    for attempt in range(5):
        tick = mt5.symbol_info_tick(SYMBOL)
        if tick and tick.bid > 0 and tick.ask > 0:
            price = tick.ask if direction == "BUY" else tick.bid
            if take_profit is not None:
                calculated_tp = take_profit
            else:
                positions = [p for p in (mt5.positions_get(symbol=SYMBOL) or []) if p.magic == magic]
                pips_to_use = DYNAMIC_PROFIT_PIPS if len(positions) >= DYNAMIC_POSITIONS_TRIGGER else PROFIT_PIPS
                calculated_tp = tp_price(direction, price, pips_to_use)
            break
        log.warning(f"Market Order: Attempt {attempt+1}/5: No valid tick for {direction} ({mt5_comment}). Retrying...")
        time.sleep(0.2)
    else:
        log.error(f"Market Order: Failed to get valid tick for {direction} ({mt5_comment}). Cannot send.")
        return False

    if price == 0.0:
        log.error(f"Market Order: Price for {SYMBOL} is zero. Cannot send {direction} ({mt5_comment}).")
        return False

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": adjusted_lot,
        "type": mt5.ORDER_TYPE_BUY if direction == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": price,
        "tp": calculated_tp,
        "sl": stop_loss,
        "deviation": SLIPPAGE,
        "magic": magic,
        "comment": mt5_comment,
        "type_time": mt5.ORDER_TIME_GTC,
    }

    s_info = mt5.symbol_info(SYMBOL)
    if s_info and hasattr(s_info, 'filling_modes'):
        allowed_filling_types = s_info.filling_modes
        if mt5.ORDER_FILLING_IOC in allowed_filling_types:
            request["type_filling"] = mt5.ORDER_FILLING_IOC
        elif mt5.ORDER_FILLING_FOK in allowed_filling_types:
            request["type_filling"] = mt5.ORDER_FILLING_FOK
        elif len(allowed_filling_types) > 0:
             request["type_filling"] = allowed_filling_types[0]
    else:
        log.warning(f"Market Order: Could not determine filling modes for {SYMBOL}. Using default.")

    log.info(f"Sending Market Order: {direction} {adjusted_lot} {SYMBOL} @ Market (Ref: {price:.5f}) TP: {calculated_tp:.5f} SL: {stop_loss:.5f} Magic: {magic} Comment: {mt5_comment}")
    result = mt5.order_send(request)

    if result is None:
        log.error(f"Market Order send failed for {direction} {adjusted_lot} ({mt5_comment}). MT5 returned None. Error: {mt5.last_error()}")
        return False

    if result.retcode == mt5.TRADE_RETCODE_DONE:
        log.info(f"🟢 Market Order: {direction} {result.volume:.2f} @ {result.price:.5f} TP {calculated_tp:.5f} (Magic: {magic}, Ticket: {result.order}, Comment: {mt5_comment}) successfully placed.")
        return True
    else:
        log.error(f"🔴 Market Order send failed for {direction} {adjusted_lot} ({mt5_comment}). Retcode: {result.retcode}, MT5 Comment: {result.comment}, Error: {mt5.last_error()}")
        if result.retcode == 10026:
            log.critical("CRITICAL: Autotrading is disabled on the server. Please enable it in MT5 Terminal (Tools -> Options -> Expert Advisors -> Allow automated trading).")
        return False

# --- Trade execution (Limit Orders) ---
def place_limit_order(direction: str, price_level: float, lot: float, magic: int, grid_name: str, action_comment_str: str = "grid_limit") -> mt5.OrderSendResult | None:
    mt5_comment = format_mt5_comment(grid_name, action_comment_str)

    adjusted_lot = adjust_lot(lot)
    symbol_info_vol_min = getattr(mt5.symbol_info(SYMBOL), "volume_min", 0.01)
    if adjusted_lot < symbol_info_vol_min:
        log.warning(f"Limit Order: Adjusted lot {adjusted_lot} is below minimum {symbol_info_vol_min} for {direction} at {price_level:.5f}. Order not placed.")
        return None

    positions = [p for p in (mt5.positions_get(symbol=SYMBOL) or []) if p.magic == magic]
    pips_to_use = DYNAMIC_PROFIT_PIPS if len(positions) >= DYNAMIC_POSITIONS_TRIGGER else PROFIT_PIPS
    limit_order_tp = tp_price(direction, price_level, pips_to_use)
    order_type = mt5.ORDER_TYPE_BUY_LIMIT if direction == "BUY" else mt5.ORDER_TYPE_SELL_LIMIT

    log.debug(f"Limit Order: Proceeding to place {direction} limit at {price_level:.5f} (strict validation removed).")

    request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": SYMBOL,
        "volume": adjusted_lot,
        "type": order_type,
        "price": price_level,
        "tp": limit_order_tp,
        "sl": 0.0,
        "magic": magic,
        "comment": mt5_comment,
        "type_time": mt5.ORDER_TIME_GTC,
    }

    if PENDING_ORDER_EXPIRATION_MIN > 0:
        expiration_time = int(time.time()) + PENDING_ORDER_EXPIRATION_MIN * 60
        request["type_time"] = mt5.ORDER_TIME_SPECIFIED
        request["expiration"] = expiration_time

    s_info = mt5.symbol_info(SYMBOL)
    if s_info and hasattr(s_info, 'filling_modes'):
        allowed_filling_types = s_info.filling_modes
        if mt5.ORDER_FILLING_RETURN in allowed_filling_types:
             request["type_filling"] = mt5.ORDER_FILLING_RETURN
        elif len(allowed_filling_types) > 0:
             request["type_filling"] = allowed_filling_types[0]

    log.info(f"Placing Limit Order: {direction} {adjusted_lot} {SYMBOL} @ {price_level:.5f} TP: {limit_order_tp:.5f} Magic: {magic} Comment: {mt5_comment}")
    result = mt5.order_send(request)

    if result is None:
        log.error(f"Limit Order placement failed (MT5 returned None) for {direction} {adjusted_lot} @ {price_level:.5f} ({mt5_comment}). Error: {mt5.last_error()}")
        return None

    if result.retcode != mt5.TRADE_RETCODE_DONE:
        current_tick_info = mt5.symbol_info_tick(SYMBOL)
        current_bid = current_tick_info.bid if current_tick_info and hasattr(current_tick_info, 'bid') else "N/A"
        current_ask = current_tick_info.ask if current_tick_info and hasattr(current_tick_info, 'ask') else "N/A"
        log.error(f"🔴 Limit Order placement FAILED for {direction} {adjusted_lot} @ {price_level:.5f} ({mt5_comment}). Retcode: {result.retcode}, MT5 Comment: {result.comment}, Error: {mt5.last_error()}. Current Bid: {current_bid}, Ask: {current_ask}")
    else:
        log.info(f"🟢 Limit Order: {direction} {result.volume:.2f} {SYMBOL} @ {result.price:.5f} TP {limit_order_tp:.5f} (Magic: {magic}, Order Ticket: {result.order}, Comment: {mt5_comment}) successfully placed.")

    return result

def cancel_pending_orders_by_side(direction: str, magic: int | None = None):
    log.info(f"Attempting to cancel all pending {direction} limit orders for magic {magic if magic is not None else 'ALL'}.")
    order_type_to_cancel = mt5.ORDER_TYPE_BUY_LIMIT if direction == "BUY" else mt5.ORDER_TYPE_SELL_LIMIT

    pending_orders = mt5.orders_get(symbol=SYMBOL) or []
    cancelled_count = 0
    for order in pending_orders:
        if (magic is None or order.magic == magic) and order.type == order_type_to_cancel:
            del_request = {"action": mt5.TRADE_ACTION_REMOVE, "order": order.ticket, "symbol": SYMBOL}
            del_res = mt5.order_send(del_request)
            if del_res and del_res.retcode == mt5.TRADE_RETCODE_DONE:
                log.info(f"Cancelled pending {direction} limit order {order.ticket} for magic {order.magic}")
                cancelled_count += 1
            else:
                err_code = del_res.retcode if del_res else "N/A"
                log.error(f"Failed to cancel pending {direction} limit order {order.ticket} (magic {order.magic}). Error: {mt5.last_error()}, Retcode: {err_code}")
    if cancelled_count > 0:
        log.info(f"Cancelled {cancelled_count} pending {direction} limit orders.")
    else:
        log.info(f"No pending {direction} limit orders found for magic {magic} to cancel.")


def sync_all_tps(direction: str, magic: int):
    log.debug(f"Attempting to sync TPs for {direction} side for magic {magic}.")
    side_to_sync = mt5.ORDER_TYPE_BUY if direction == "BUY" else mt5.ORDER_TYPE_SELL

    all_positions_of_side = [p for p in (mt5.positions_get(symbol=SYMBOL) or []) if p.magic == magic and p.type == side_to_sync and "newgridhedge" not in p.comment]

    if not all_positions_of_side:
        log.debug(f"No open {direction} positions for magic {magic} to sync TPs for.")
        return

    try:
        all_positions_of_side.sort(key=lambda p: (p.time_msc if hasattr(p, 'time_msc') and p.time_msc > 0 else p.time, p.ticket), reverse=True)
        newest_position = all_positions_of_side[0]
    except Exception as e:
        log.error(f"Could not determine newest position for {direction} side (magic {magic}) for TP sync: {e}")
        return

    if newest_position.tp == 0:
        log.debug(f"Newest {direction} position {newest_position.ticket} (magic {magic}) has no TP. Nothing to sync.")
        return

    new_tp_level = newest_position.tp
    log.debug(f"TP Sync check for {direction} (magic {magic}): Newest position {newest_position.ticket} TP is {new_tp_level:.5f}. Comparing with others.")
    synced_count = 0

    mt5_comment_sync = format_mt5_comment(grid_states[magic]['name'], f"sync_tp_{newest_position.ticket}")

    for p in all_positions_of_side:
        if p.ticket == newest_position.ticket or math.isclose(p.tp, new_tp_level, abs_tol=mt5.symbol_info(SYMBOL).point * 0.1):
            continue

        log.info(f"TP Sync for {direction} (magic {magic}): Position {p.ticket} (TP: {p.tp:.5f}) will be synced to {new_tp_level:.5f}")
        modify_request = {
            "action": mt5.TRADE_ACTION_SLTP,
            "symbol": SYMBOL,
            "position": p.ticket,
            "tp": new_tp_level,
            "sl": p.sl,
            "magic": magic,
            "comment": mt5_comment_sync
        }
        result = mt5.order_send(modify_request)
        if result and result.retcode == mt5.TRADE_RETCODE_DONE:
            log.info(f"🔄 TP synced for {direction} position {p.ticket} to {new_tp_level:.5f}")
            synced_count += 1
        else:
            err_code = result.retcode if result else "N/A"
            last_err = mt5.last_error()
            log.warning(f"sync_all_tps: Failed to sync TP for {direction} position {p.ticket}. Error: {last_err}, Retcode: {err_code}")

    if synced_count > 0:
        log.info(f"✅ TP Sync Summary for {direction} (magic {magic}): {synced_count} positions updated to TP {new_tp_level:.5f}")
    elif newest_position:
        log.debug(f"No TPs needed syncing for {direction} side (magic {magic}) based on newest position {newest_position.ticket}.")


# --- Grid recalculation ---
def recalc_grid(magic: int):
    global grid_states

    grid_state = grid_states.get(magic)
    if not grid_state:
        log.warning(f"Recalc: Could not find state for magic number {magic}")
        return

    positions = [p for p in (mt5.positions_get(symbol=SYMBOL) or []) if p.magic == magic]
    buy_positions = [p for p in positions if p.type == mt5.ORDER_TYPE_BUY]
    sell_positions = [p for p in positions if p.type == mt5.ORDER_TYPE_SELL]
    s_info_digits = mt5.symbol_info(SYMBOL).digits

    if buy_positions:
        num_for_dyn_trigger_buy = len(buy_positions)
        current_grid_step_pips_buy = DYNAMIC_STEP_PIPS if num_for_dyn_trigger_buy >= DYNAMIC_POSITIONS_TRIGGER else GRID_STEP_PIPS
        step_in_price_buy = current_grid_step_pips_buy * pip_val()
        min_buy_price = min(p.price_open for p in buy_positions)
        grid_state["next_buy_px"] = round(min_buy_price - step_in_price_buy, s_info_digits)
        log.debug(f"Recalc (Magic {magic}): Next BUY LIMIT target calculated: {grid_state['next_buy_px']:.{s_info_digits}f}")
    else:
        grid_state["next_buy_px"] = None
        log.debug(f"Recalc (Magic {magic}): No open buy positions, next_buy_px is None.")

    if sell_positions:
        num_for_dyn_trigger_sell = len(sell_positions)
        current_grid_step_pips_sell = DYNAMIC_STEP_PIPS if num_for_dyn_trigger_sell >= DYNAMIC_POSITIONS_TRIGGER else GRID_STEP_PIPS
        step_in_price_sell = current_grid_step_pips_sell * pip_val()
        max_sell_price = max(p.price_open for p in sell_positions)
        grid_state["next_sell_px"] = round(max_sell_price + step_in_price_sell, s_info_digits)
        log.debug(f"Recalc (Magic {magic}): Next SELL LIMIT target calculated: {grid_state['next_sell_px']:.{s_info_digits}f}")
    else:
        grid_state["next_sell_px"] = None
        log.debug(f"Recalc (Magic {magic}): No open sell positions, next_sell_px is None.")

    if grid_state["next_buy_px"] is not None and grid_state["next_sell_px"] is not None:
         log.info(f"📐 Grid levels for magic {magic}: Next BUY ≤ {grid_state['next_buy_px']:.{s_info_digits}f}, Next SELL ≥ {grid_state['next_sell_px']:.{s_info_digits}f}")
    else:
        log.debug(f"📐 Grid levels for magic {magic} are partially or fully None.")

# --- Auto-hedge handling (using Market Orders) ---
def hedge_if_empty():
    global grid_states, next_magic_number

    positions = mt5.positions_get(symbol=SYMBOL) or []
    if positions:
        return

    log.info("🔄 No positions found. Initiating initial grid (Magic: %d)...", base_magic_number)

    buy_success = send_market_order("BUY", LOT_SMALL, base_magic_number, "A", "initial_hedge_buy")
    sell_success = send_market_order("SELL", LOT_SMALL, base_magic_number, "A", "initial_hedge_sell")

    if buy_success and sell_success:
        # Create state for the new base grid
        grid_states[base_magic_number] = {
            "name": "A",
            "next_buy_px": None, "next_sell_px": None,
            "buy_sequence_index": 0, "sell_sequence_index": 0,
            "prev_buy_count": 1, "prev_sell_count": 1,
            "capped_buy": False, "capped_sell": False
        }
        log.info("Initial market hedge orders placed successfully for Grid A (magic %d).", base_magic_number)
        recalc_grid(base_magic_number)
        time.sleep(OPEN_DELAY)
    else:
        log.error("Failed to place one or both initial market hedge orders.")


# --- Closed hedge detection (using Market Orders) ---
def handle_closed_hedge(magic: int):
    global grid_states

    grid_state = grid_states.get(magic)
    if not grid_state: return

    all_positions = mt5.positions_get(symbol=SYMBOL) or []
    positions_in_grid = [p for p in all_positions if p.magic == magic]

    current_buy_count = sum(1 for p in positions_in_grid if p.type == mt5.ORDER_TYPE_BUY)
    current_sell_count = sum(1 for p in positions_in_grid if p.type == mt5.ORDER_TYPE_SELL)

    if current_buy_count < grid_state.get("prev_buy_count", current_buy_count + 1):
        log.info(f"🔔 BUY position closed in grid {grid_state['name']} (magic {magic}). Resetting BUY side.")
        cancel_pending_orders_by_side("BUY", magic)
        grid_state["buy_sequence_index"] = 0
        grid_state["next_buy_px"] = None
        grid_state["capped_buy"] = False # Un-cap the side
        if current_buy_count == 0 and current_sell_count > 0:
            if send_market_order("BUY", LOT_SMALL, magic, grid_state['name'], action_comment_str=f"re_hedge_buy"):
                 time.sleep(OPEN_DELAY)
        recalc_grid(magic)

    if current_sell_count < grid_state.get("prev_sell_count", current_sell_count + 1):
        log.info(f"🔔 SELL position closed in grid {grid_state['name']} (magic {magic}). Resetting SELL side.")
        cancel_pending_orders_by_side("SELL", magic)
        grid_state["sell_sequence_index"] = 0
        grid_state["next_sell_px"] = None
        grid_state["capped_sell"] = False # Un-cap the side
        if current_sell_count == 0 and current_buy_count > 0:
            if send_market_order("SELL", LOT_SMALL, magic, grid_state['name'], action_comment_str=f"re_hedge_sell"):
                time.sleep(OPEN_DELAY)
        recalc_grid(magic)

    grid_state["prev_buy_count"] = current_buy_count
    grid_state["prev_sell_count"] = current_sell_count

    # After all handling, if a grid is now completely empty, remove it.
    if current_buy_count == 0 and current_sell_count == 0:
        log.warning(f"Grid {grid_state['name']} (magic {magic}) is now empty. Deactivating this grid.")
        cancel_pending_orders_by_side("BUY", magic)
        cancel_pending_orders_by_side("SELL", magic)
        if magic in grid_states:
            del grid_states[magic]


# --- Grid stepping logic (using Limit Orders) ---
def step_grid(magic: int):
    global grid_states

    grid_state = grid_states.get(magic)
    if not grid_state: return

    all_positions = mt5.positions_get(symbol=SYMBOL) or []
    if len(all_positions) >= MAX_POSITIONS:
        log.warning(f"Max open positions ({MAX_POSITIONS}) reached. No new grid limit orders will be placed.")
        return

    # Unpack state for this grid
    next_buy_px = grid_state.get("next_buy_px")
    next_sell_px = grid_state.get("next_sell_px")
    buy_sequence_index = grid_state.get("buy_sequence_index", 0)
    sell_sequence_index = grid_state.get("sell_sequence_index", 0)

    if next_buy_px is None or next_sell_px is None:
        log.debug(f"step_grid (Magic {magic}): One or both grid levels are None. Attempting recalc.")
        recalc_grid(magic)
        next_buy_px = grid_state.get("next_buy_px")
        next_sell_px = grid_state.get("next_sell_px")

    s_info = mt5.symbol_info(SYMBOL)
    if not s_info: return
    symbol_point = s_info.point
    s_info_digits = s_info.digits
    abs_tolerance_for_isclose = symbol_point * 2.0

    pending_orders = [o for o in (mt5.orders_get(symbol=SYMBOL) or []) if o.magic == magic]
    current_pending_buy_limits = sum(1 for o in pending_orders if o.type == mt5.ORDER_TYPE_BUY_LIMIT)
    current_pending_sell_limits = sum(1 for o in pending_orders if o.type == mt5.ORDER_TYPE_SELL_LIMIT)

    # --- BUY LIMIT ---
    if grid_state.get('capped_buy', False):
        log.debug(f"step_grid (Magic {magic}): BUY side is capped. Skipping.")
    elif next_buy_px is not None and current_pending_buy_limits < MAX_PENDING_GRID_ORDERS_PER_SIDE:
        if buy_sequence_index < len(GRID_LOT_SEQUENCE):
            raw_lot = GRID_LOT_SEQUENCE[buy_sequence_index]
        else:
            raw_lot = GRID_LOT_SEQUENCE[-1]
        current_lot = adjust_lot(raw_lot)

        for i in range(INVALID_PRICE_RETRY_LIMIT):
            order_result = place_limit_order("BUY", next_buy_px, current_lot, magic, grid_state['name'], "gridbuy")
            if order_result and order_result.retcode == mt5.TRADE_RETCODE_DONE:
                grid_state["buy_sequence_index"] += 1
                recalc_grid(magic)
                time.sleep(OPEN_DELAY)
                break
            elif order_result and order_result.retcode == 10015: # Invalid price
                log.warning(f"BUY_LIMIT failed due to Invalid Price (Retry {i+1}/{INVALID_PRICE_RETRY_LIMIT}) for magic {magic}. Waiting 1s.")
                time.sleep(1)
            else: # Other error
                log.info("Attempt to place BUY_LIMIT order was not successful (see previous logs for reason).")
                break
        else: # This block runs if the for loop completes without breaking
            log.warning(f"BUY_LIMIT retry limit reached for magic {magic}. Attempting market order.")
            if send_market_order("BUY", current_lot, magic, grid_state['name'], "grid_buy_market_fallback"):
                grid_state["buy_sequence_index"] += 1
                recalc_grid(magic)

    # --- SELL LIMIT ---
    if grid_state.get('capped_sell', False):
        log.debug(f"step_grid (Magic {magic}): SELL side is capped. Skipping.")
    elif next_sell_px is not None and current_pending_sell_limits < MAX_PENDING_GRID_ORDERS_PER_SIDE:
        if sell_sequence_index < len(GRID_LOT_SEQUENCE):
            raw_lot = GRID_LOT_SEQUENCE[sell_sequence_index]
        else:
            raw_lot = GRID_LOT_SEQUENCE[-1]
        current_lot = adjust_lot(raw_lot)

        for i in range(INVALID_PRICE_RETRY_LIMIT):
            order_result = place_limit_order("SELL", next_sell_px, current_lot, magic, grid_state['name'], "gridsell")
            if order_result and order_result.retcode == mt5.TRADE_RETCODE_DONE:
                grid_state["sell_sequence_index"] += 1
                recalc_grid(magic)
                time.sleep(OPEN_DELAY)
                break
            elif order_result and order_result.retcode == 10015: # Invalid price
                log.warning(f"SELL_LIMIT failed due to Invalid Price (Retry {i+1}/{INVALID_PRICE_RETRY_LIMIT}) for magic {magic}. Waiting 1s.")
                time.sleep(1)
            else:
                log.info("Attempt to place SELL_LIMIT order was not successful (see previous logs for reason).")
                break
        else:
            log.warning(f"SELL_LIMIT retry limit reached for magic {magic}. Attempting market order.")
            if send_market_order("SELL", current_lot, magic, grid_state['name'], "grid_sell_market_fallback"):
                grid_state["sell_sequence_index"] += 1
                recalc_grid(magic)


# --- New Grid Trigger Logic ---
def check_and_create_new_grid():
    global grid_states, next_magic_number

    if len(grid_states) >= MAX_ACTIVE_GRIDS:
        log.debug(f"Max active grids ({MAX_ACTIVE_GRIDS}) reached. Not creating new grid.")
        return

    for magic in list(grid_states.keys()):
        grid_state = grid_states.get(magic)
        if not grid_state: continue

        positions = [p for p in (mt5.positions_get(symbol=SYMBOL) or []) if p.magic == magic]
        buy_positions = [p for p in positions if p.type == mt5.ORDER_TYPE_BUY]
        sell_positions = [p for p in positions if p.type == mt5.ORDER_TYPE_SELL]

        log.debug(f"Checking trigger for magic {magic}: Buys={len(buy_positions)}, Sells={len(sell_positions)}, Trigger={DYNAMIC_POSITIONS_TRIGGER}")

        # Check BUY side trigger
        if not grid_state.get("capped_buy", False) and len(buy_positions) >= DYNAMIC_POSITIONS_TRIGGER:
            log.warning(f"🚨 DYNAMIC TRIGGER HIT for BUY side on grid {grid_state['name']} (magic {magic}). Creating new grid.")

            total_buy_volume = sum(p.volume for p in buy_positions)
            hedge_lot = adjust_lot(total_buy_volume / 2.0)

            buy_positions.sort(key=lambda p: p.time, reverse=True)
            new_sl = buy_positions[0].tp if buy_positions else 0.0
            log.info(f"Total buy volume for grid {grid_state['name']} is {total_buy_volume:.2f}. Counter-hedging with SELL lot {hedge_lot:.2f} and SL {new_sl:.5f}")

            new_magic = next_magic_number
            new_grid_name = chr(ord(grid_state['name']) + 1)

            if send_market_order("SELL", hedge_lot, new_magic, new_grid_name, action_comment_str=f"newgrid_hedge_from_{magic}", stop_loss=new_sl, take_profit=0.0): # Set No TP
                if send_market_order("BUY", LOT_SMALL, new_magic, new_grid_name, action_comment_str=f"newgrid_hedge_from_{magic}"):
                    log.info(f"✅ Successfully created new independent grid {new_grid_name} with magic number {new_magic}.")
                    grid_states[new_magic] = {
                        "name": new_grid_name,
                        "buy_sequence_index": 0, "sell_sequence_index": 0,
                        "prev_buy_count": 1, "prev_sell_count": 1,
                        "capped_buy": False, "capped_sell": False,
                    }
                    recalc_grid(new_magic)
                    next_magic_number += 1

                    log.info(f"Updating TPs for original grid {grid_state['name']} (magic {magic}) to DYNAMIC_PROFIT_PIPS.")
                    for p in buy_positions:
                        new_tp = tp_price("BUY", p.price_open, DYNAMIC_PROFIT_PIPS)
                        mt5.order_send({"action": mt5.TRADE_ACTION_SLTP, "position": p.ticket, "tp": new_tp, "magic": magic})

                    cancel_pending_orders_by_side("BUY", magic)
                    grid_state['capped_buy'] = True
                    grid_state['next_buy_px'] = None
            else:
                log.error(f"Failed to create new hedge grid from grid {magic}.")

        # Check SELL side trigger
        if not grid_state.get("capped_sell", False) and len(sell_positions) >= DYNAMIC_POSITIONS_TRIGGER:
            log.warning(f"🚨 DYNAMIC TRIGGER HIT for SELL side on grid {grid_state['name']} (magic {magic}). Creating new grid.")
            total_sell_volume = sum(p.volume for p in sell_positions)
            hedge_lot = adjust_lot(total_sell_volume / 2.0)

            sell_positions.sort(key=lambda p: p.time, reverse=True)
            new_sl = sell_positions[0].tp if sell_positions else 0.0
            log.info(f"Total sell volume for grid {grid_state['name']} is {total_sell_volume:.2f}. Counter-hedging with BUY lot {hedge_lot:.2f} and SL {new_sl:.5f}")

            new_magic = next_magic_number
            new_grid_name = chr(ord(grid_state['name']) + 1)

            if send_market_order("BUY", hedge_lot, new_magic, new_grid_name, action_comment_str=f"newgrid_hedge_from_{magic}", stop_loss=new_sl, take_profit=0.0): # Set No TP
                if send_market_order("SELL", LOT_SMALL, new_magic, new_grid_name, action_comment_str=f"newgrid_hedge_from_{magic}"):
                    log.info(f"✅ Successfully created new independent grid {new_grid_name} with magic number {new_magic}.")
                    grid_states[new_magic] = {
                        "name": new_grid_name,
                        "buy_sequence_index": 0, "sell_sequence_index": 0,
                        "prev_buy_count": 1, "prev_sell_count": 1,
                        "capped_buy": False, "capped_sell": False,
                    }
                    recalc_grid(new_magic)
                    next_magic_number += 1

                    log.info(f"Updating TPs for original grid {magic} to DYNAMIC_PROFIT_PIPS.")
                    for p in sell_positions:
                        new_tp = tp_price("SELL", p.price_open, DYNAMIC_PROFIT_PIPS)
                        mt5.order_send({"action": mt5.TRADE_ACTION_SLTP, "position": p.ticket, "tp": new_tp, "magic": magic})

                    cancel_pending_orders_by_side("SELL", magic)
                    grid_state['capped_sell'] = True
                    grid_state['next_sell_px'] = None


# --- Main execution loop ---
def run():
    global initial_equity, loop_counter, grid_states, next_magic_number

    mt5_login()

    existing_positions = mt5.positions_get(symbol=SYMBOL) or []
    if existing_positions:
        magics = sorted(list(set(p.magic for p in existing_positions)))
        log.info(f"↻ Restarted. Found existing positions with magic numbers: {magics}")
        for i, magic in enumerate(magics):
            grid_name = chr(ord('A') + i)
            positions_in_grid = [p for p in existing_positions if p.magic == magic]
            buy_positions = [p for p in positions_in_grid if p.type == mt5.ORDER_TYPE_BUY]
            sell_positions = [p for p in positions_in_grid if p.type == mt5.ORDER_TYPE_SELL]

            buy_grid_trades = sum(1 for p in buy_positions if "gridbuy" in p.comment or "marketfallback" in p.comment)
            sell_grid_trades = sum(1 for p in sell_positions if "gridsell" in p.comment or "marketfallback" in p.comment)

            grid_states[magic] = {
                "name": grid_name,
                "buy_sequence_index": buy_grid_trades,
                "sell_sequence_index": sell_grid_trades,
                "next_buy_px": None,
                "next_sell_px": None,
                "prev_buy_count": len(buy_positions),
                "prev_sell_count": len(sell_positions),
                "capped_buy": False,
                "capped_sell": False,
            }
            log.info(f"Grid {grid_name} (magic {magic}) reconstructed: Buys={len(buy_positions)}, Sells={len(sell_positions)}, Next Buy Lot Index={buy_grid_trades}, Next Sell Lot Index={sell_grid_trades}")
            recalc_grid(magic)

        if magics:
            next_magic_number = max(magics) + 1
    else:
        hedge_if_empty()

    try:
        while True:
            loop_counter += 1
            if mt5.terminal_info() is None:
                log.error("Lost connection to MetaTrader 5 terminal. Attempting to reconnect...")
                mt5.shutdown()
                time.sleep(10)
                mt5_login()
                if mt5.terminal_info() is None:
                    log.error("Failed to reconnect. Exiting.")
                    break

            if loop_counter % LOG_BALANCE_INTERVAL == 0:
                acc_info = mt5.account_info()
                if acc_info:
                    log.info(f"Account Status - Balance: {acc_info.balance:.2f} {acc_info.currency}, Equity: {acc_info.equity:.2f} {acc_info.currency}")

            if PROFIT_TARGET_AMT > 0:
                account_info = mt5.account_info()
                if account_info:
                    current_profit = account_info.equity - initial_equity
                    log.debug(f"Profit Check: Equity={account_info.equity:.2f}, InitialEquity={initial_equity:.2f}, Profit={current_profit:.2f}, Target={PROFIT_TARGET_AMT:.2f}")
                    if current_profit >= PROFIT_TARGET_AMT:
                        log.warning(f"🎯 PROFIT TARGET of {PROFIT_TARGET_AMT:.2f} REACHED! Current profit: {current_profit:.2f}. Equity: {account_info.equity:.2f}")
                        log.warning(f"Closing all positions and pending orders for symbol {SYMBOL}...")

                        cancel_pending_orders_by_side("BUY", None)
                        cancel_pending_orders_by_side("SELL", None)
                        log.info(f"Profit Target: Cancelled all pending grid orders.")

                        open_positions = mt5.positions_get(symbol=SYMBOL) or []
                        closed_positions_count = 0
                        for position in open_positions:
                            close_direction = mt5.ORDER_TYPE_SELL if position.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                            close_tick = mt5.symbol_info_tick(SYMBOL)
                            if not close_tick:
                                log.error(f"Could not get tick to close position {position.ticket}. Skipping.")
                                continue
                            close_price = close_tick.bid if position.type == mt5.ORDER_TYPE_BUY else close_tick.ask
                            if close_price == 0:
                                log.error(f"Invalid close price (0) for position {position.ticket}. Skipping.")
                                continue

                            close_request = {
                                "action": mt5.TRADE_ACTION_DEAL, "symbol": SYMBOL,
                                "volume": position.volume, "type": close_direction,
                                "position": position.ticket, "price": close_price,
                                "deviation": 30, "magic": position.magic,
                                "comment": format_mt5_comment("System", "profit_target_close")
                            }
                            result = mt5.order_send(close_request)
                            if result and result.retcode == mt5.TRADE_RETCODE_DONE:
                                log.info(f"Closed position {position.ticket} for profit target.")
                                closed_positions_count +=1
                            else:
                                log.error(f"Failed to close position {position.ticket}. Error: {mt5.last_error()}, Retcode: {result.retcode if result else 'N/A'}")
                                if result and result.retcode == 10026:
                                    log.critical("CRITICAL: Autotrading is disabled on the server. Please enable it in MT5 Terminal (Tools -> Options -> Expert Advisors -> Allow automated trading).")


                        log.info(f"Profit target: Closed {closed_positions_count}/{len(open_positions)} open positions.")

                        log.info("Resetting bot state for new cycle...")
                        grid_states = {} # Clear all grids

                        new_account_info = mt5.account_info()
                        if new_account_info:
                            initial_equity = new_account_info.equity
                            log.info(f"New initial_equity for profit tracking: {initial_equity:.2f}")
                        else:
                            log.error("Could not get account info to update initial_equity after profit target reset!")

                        hedge_if_empty()
                        log.info("Bot reset complete due to profit target. Continuing operation.")
                        continue

            check_and_create_new_grid()

            for magic in list(grid_states.keys()):
                handle_closed_hedge(magic)
                step_grid(magic)
                sync_all_tps("BUY", magic)
                sync_all_tps("SELL", magic)

            hedge_if_empty()

            time.sleep(LOOP_MS / 1000.0)

    except KeyboardInterrupt:
        log.info("User requested shutdown (KeyboardInterrupt).")
    except Exception as e:
        log.error(f"An unexpected error occurred in the main loop: {e}")
        log.error(traceback.format_exc())
    finally:
        log.info("Shutting down MetaTrader 5 connection...")
        cancel_pending_orders_by_side("BUY", None)
        cancel_pending_orders_by_side("SELL", None)
        mt5.shutdown()
        log.info("Bot stopped.")

if __name__ == '__main__':
    log.info(f"Starting Grid Martingale Bot (v110.1 - Grid Naming)")
    log.info(f"SYMBOL: {SYMBOL}, LOT_SMALL: {LOT_SMALL}")
    log.info(f"GRID LOT SEQUENCE: {GRID_LOT_SEQUENCE}")
    log.info(f"GRID_PIPS: {GRID_STEP_PIPS}, DYN_GRID_PIPS: {DYNAMIC_STEP_PIPS}, PROFIT_PIPS: {PROFIT_PIPS}, DYN_PROFIT_PIPS: {DYNAMIC_PROFIT_PIPS}")
    log.info(f"MAX_POS: {MAX_POSITIONS}, OPEN_DELAY: {OPEN_DELAY}s, DYN_TRIGGER: {DYNAMIC_POSITIONS_TRIGGER}, PROFIT_TARGET: {PROFIT_TARGET_AMT}")
    log.info(f"PENDING_ORDER_EXP_MIN: {PENDING_ORDER_EXPIRATION_MIN}, MAX_PENDING_GRID_ORDERS_PER_SIDE: {MAX_PENDING_GRID_ORDERS_PER_SIDE}")
    log.info(f"LOG_BALANCE_INTERVAL: {LOG_BALANCE_INTERVAL} loops")
    run()